In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Input, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model

import os, glob
import time


In [ ]:
from tensorflow.keras.models import load_model

model_path = '/kaggle/input/load-model/my_best_center_crop_resnet152v2_model.h5'
model = load_model(model_path)
name = 'centercrop'

In [ ]:
# test_dir
test_dir='/kaggle/input/early-anthracnose-for-test/early_anthracnose'

# test_ds
ds_health =glob.glob('/kaggle/input/early-anthracnose-for-test/early_anthracnose/healthy/*') 
ds_anth =glob.glob('/kaggle/input/early-anthracnose-for-test/early_anthracnose/anthracnose/*')

# test_df
test_path = ds_health + ds_anth
test_label = ["healthy" for _ in range(len(ds_health))] + ["anthracnose" for _ in range(len(ds_anth))]
test_df = pd.DataFrame({"path":test_path, "label":test_label})

# Define data preprocessing and data_generator

def create_data_generators(test_df, image_size, batch_size):

    # Define a function to crop the image
    def crop_image(img):
        # get the shape of the input image
        h, w, _ = img.shape
        

        # set the desired crop size
        crop_size = 224

        # calculate the starting point of the crop
        start_y = int((h - crop_size) / 2)
        start_x = int((w - crop_size) / 2)

        # crop the image
        crop_img = img[start_y:start_y + crop_size, start_x:start_x + crop_size, :]

        return crop_img

   # Define the ImageDataGenerator with only rescaling for test data
    test_generator = ImageDataGenerator(
        horizontal_flip=True,
        rotation_range=20,
        rescale=1/255.0,
        preprocessing_function=crop_image)

   # Flow from DataFrame using test_generator for test data
    test_generator_iterator = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='path',
        y_col='label',
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=False)
    test_batch, test_label_batch = next(test_generator_iterator)

    return test_generator_iterator,test_batch


# Call the function to create data generators
test_gen_iter,test_batch  = create_data_generators(test_df=test_df,image_size=224, batch_size=8)


## model.evaluate()

In [ ]:
loss0, accuracy0 = model.evaluate(test_gen_iter)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the label encoder to the label column
le.fit(test_df['label'])

# Transform the label column to encoded labels
true_labels = le.transform(test_df['label'])

# test_df['label']와 predicted_labels 비교
predicted_labels = model.predict(test_gen_iter)
predicted_labels = np.argmax(predicted_labels, axis=1)  # 모델의 출력을 가장 높은 확률을 가진 클래스로 변환


## Classification Report

In [ ]:

import csv
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def generate_classification_report(model,name):   
    
    with open(name+'_report.csv', 'w') as file:
        writer = csv.writer(file)
        for line in report.split('\n'):
            writer.writerow(line.split(','))
        
        evaluation_results = model.evaluate(test_gen_iter)
        evaluation_results_header = ['Metric', 'Value']
            
        writer.writerow(['Loss', evaluation_results[0]])
        writer.writerow(['Accuracy', evaluation_results[1]])
        # 정확도(accuracy)를 계산합니다.
        accuracy = accuracy_score(true_labels, predicted_labels)
        writer.writerow(['Accuracy_score', accuracy])

        # 정밀도(precision)를 계산합니다.
        precision = precision_score(true_labels, predicted_labels,average='micro')
        writer.writerow(['Precision', precision])

        # 재현율(recall)를 계산합니다.
        recall = recall_score(true_labels, predicted_labels,average='micro')
        writer.writerow(['Recall', recall])

        # F1-스코어(F1-score)를 계산합니다.
        f1 = f1_score(true_labels, predicted_labels,average='micro')
        writer.writerow(['F1_score', f1])

        # 결과를 출력합니다.
        print("정확도: {:.4f}".format(accuracy))
        print("정밀도: {:.4f}".format(precision))
        print("재현율: {:.4f}".format(recall))
        print("F1-스코어: {:.4f}".format(f1))
        
generate_classification_report(model=model,name=name)

## Confusion Matrix

In [ ]:
# Define confusion matrix

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

def generate_confusion_matrix(model,name):
    # create a LabelBinarizer object
   

    # Create a LabelEncoder object
    le = LabelEncoder()

    # Fit the label encoder to the label column
    le.fit(test_df['label'])

    # Transform the label column to encoded labels
    true_labels = le.transform(test_df['label'])

    # Transform the label column to binary list
    label_list = le.transform(test_df['label']).tolist()

    # Flatten the list of ground truth labels and convert to numpy array
    true_labels = np.ravel(label_list)
    # Use the model to predict the classes of the test data
    predicted_labels = model.predict(test_gen_iter)

    # Convert predicted labels to integer format
    predicted_labels = tf.argmax(predicted_labels, axis=1).numpy()

    # Get confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    print(cm)
    
    class_names = list(test_gen_iter.class_indices.keys())
    
    # Plot the confusion matrix
    plt.imshow(cm, cmap=plt.cm.GnBu, interpolation='nearest')
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    tick_marks = np.arange(len(class_names))-0.5
    plt.xticks(tick_marks, class_names, rotation=20)
    plt.yticks(tick_marks, class_names)
    
    plt.xlabel('Predicted Label', labelpad=25)
    plt.ylabel('True Label')

    thresh = cm.max()-20
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i,j], "d"), ha="center", va="center",
                    color="white" if cm[i,j] > thresh else "black", fontsize=16)

     # Save the figure as a JPG image
    plt.savefig(name +'_confusion_matrix.jpg', dpi=500,bbox_inches='tight')
generate_confusion_matrix(model=model,name=name)

## t-SNE

In [ ]:

from sklearn.manifold import TSNE

def extract_features(model, data):
    # 주어진 모델의 입력과 'global_average_pooling2d' 레이어의 출력을 연결하는 모델, 이를 통해 모델의 특성을 추출
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('flatten').output)
    features = feature_extractor.predict(data)
    return features

# 모델과 데이터로부터 특성 추출
features = extract_features(model,test_gen_iter)
# TSNE를 사용하여 2차원으로 차원 축소
tsne = TSNE(n_components=2,perplexity=5).fit_transform(features)

In [ ]:
# 레이블에 따라 색상 다르게

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 시각화할 데이터 포인트의 색상과 레이블 설정
colors_per_class = {
    'anthracnose': 'red',
    'healthy': 'blue'
}

# 시각화할 데이터 포인트의 색상 저장하는 리스트 초기화
scatter_colors = []

# test_df의 각 데이터 포인트에 대해 색상을 저장
for _, data_point in test_df.iterrows():
    color = colors_per_class[data_point['label']]
    scatter_colors.append(color)

# 산점도 그리기
plt.scatter(tsne[:, 0], tsne[:, 1], c=scatter_colors)

# 그래프 제목 설정
plt.title("t-SNE Visualization")

# x축, y축 레이블 설정
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")

# 범례 항목 생성
legend_patches = [mpatches.Patch(color=color, label=label) for label, color in colors_per_class.items()]

# 범례 표시
plt.legend(handles=legend_patches)

# 그래프 출력
plt.savefig(name+'_t-sne.png',dpi=500)
plt.show()


## PCA

In [ ]:
from sklearn.decomposition import PCA

# PCA를 사용하여 2차원으로 차원 축소
pca = PCA(n_components=2)
pca_result = pca.fit_transform(features)

# 레이블에 따라 색상 다르게


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 시각화할 데이터 포인트의 색상과 레이블 설정
colors_per_class = {
    'bacterial_spot': 'red',
    'powdery_mildew' : 'green',
    'PepMoV': 'purple',
    'healthy': 'blue'
}

# 시각화할 데이터 포인트의 색상 저장하는 리스트 초기화
scatter_colors = []

# test_df의 각 데이터 포인트에 대해 색상을 저장
for _, data_point in test_df.iterrows():
    color = colors_per_class[data_point['label']]
    scatter_colors.append(color)

# 산점도 그리기
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=scatter_colors)

# 그래프 제목 설정
plt.title("PCA Visualization")

# x축, y축 레이블 설정
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")

# 범례 항목 생성
legend_patches = [mpatches.Patch(color=color, label=label) for label, color in colors_per_class.items()]

# 범례 표시
plt.legend(handles=legend_patches)

# 그래프 출력
plt.savefig(name+'_pca.png',dpi=500)
plt.show()


## Grad-CAM

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

def gradcam_visualization(model, image, layer_name, class_index):
    # GradCAM을 위한 모델 생성
    grad_model = Model(inputs=model.input, outputs=(model.get_layer(layer_name).output, model.output))

    # 이미지 전처리
    img = np.expand_dims(image, axis=0)
    img = img.astype(np.float32) / 255.0

    # 특성 맵과 예측 결과 가져오기
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        loss = predictions[:, class_index]

    # 클래스에 대한 gradient 계산
    grads = tape.gradient(loss, conv_outputs)[0]

    # 특성 맵과 gradient를 가중 평균하여 heatmap 생성
    weights = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)

    # heatmap 후처리
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # heatmap을 원본 이미지 크기로 조정
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

    # heatmap을 원본 이미지에 적용하여 시각화
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

    return superimposed_img

# GradCAM 시각화 예시
class_index = 0  # 시각화할 클래스 인덱스
layer_name = 'global_average_pooling2d'  # 시각화할 레이어 이름
image = test_batch[0]  # 시각화할 이미지

visualization = gradcam_visualization(model, image, layer_name, class_index)
